<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0"> </div>
    <div style="float: left; margin-left: 10px;"> <h1>LangChain for Generative AI</h1>
<h1>LangChain</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
from collections import Counter
from pprint import pprint
from operator import itemgetter

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

import torch

import openai
from openai import OpenAI

import transformers
from transformers import pipeline
from transformers import set_seed
set_seed(42) # Set the seed to get reproducible results


import langchain
from langchain.chains import create_sql_query_chain
from langchain.tools import DuckDuckGoSearchRun

import langchain_openai
from langchain_openai import ChatOpenAI

import langchain_anthropic
from langchain_anthropic import ChatAnthropic

import langchain_core
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnablePassthrough

import langchain_community
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

import watermark

%load_ext watermark
%matplotlib inline

We start by print out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.11.7
IPython version      : 8.12.3

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 23.6.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Git hash: 0b932bf32c4e9b1e4ff9126fc7e8e7ac7b4205ff

langchain_core     : 0.2.3
langchain_openai   : 0.1.8
transformers       : 4.41.1
langchain          : 0.2.2
langchain_anthropic: 0.1.15
pandas             : 2.2.3
langchain_community: 0.2.1
torch              : 2.3.0
watermark          : 2.4.3
matplotlib         : 3.8.0
openai             : 1.30.5
numpy              : 1.26.4



Load default figure style

In [3]:
plt.style.use('./d4sci.mplstyle')

# OpenAI

The first step is generate API key on the OpenAI website and store it as the "OPENAI_API_KEY" variable in your local environment. Without it we won't be able to do anything. You can find your API key in your using settings: https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key. Then we are ready to instantiate the client

In [4]:
client = OpenAI()

In [5]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
        {
            "role": "user", 
            "content": "What was Superman's weakness?"
        },
    ]
)

In [6]:
print(response)

ChatCompletion(id='chatcmpl-AVlBKZ3AK1ebP607ObfRLejPGembN', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Superman's most well-known weakness is kryptonite, a mineral from his home planet of Krypton. Exposure to kryptonite weakens Superman and can potentially be fatal to him. Different forms of kryptonite have varied effects; for example, green kryptonite weakens him, while red kryptonite can cause unpredictable, often bizarre changes in his behavior and powers. Besides kryptonite, Superman is also vulnerable to magic, which can affect him just as it would any other being. Additionally, he's not immune to the effects of a red sun, which can drain him of his powers.", role='assistant', function_call=None, tool_calls=None, refusal=None))], created=1732132898, model='gpt-4o-2024-08-06', object='chat.completion', system_fingerprint='fp_7f6be3efb0', usage=CompletionUsage(completion_tokens=115, prompt_tokens=13, total_tokens=128, prompt_

In [7]:
print(response.choices[0].message.content)

Superman's most well-known weakness is kryptonite, a mineral from his home planet of Krypton. Exposure to kryptonite weakens Superman and can potentially be fatal to him. Different forms of kryptonite have varied effects; for example, green kryptonite weakens him, while red kryptonite can cause unpredictable, often bizarre changes in his behavior and powers. Besides kryptonite, Superman is also vulnerable to magic, which can affect him just as it would any other being. Additionally, he's not immune to the effects of a red sun, which can drain him of his powers.


# LangChain

We instantiate the LangChain interface for OpenAI

In [8]:
model = ChatOpenAI(model="gpt-4o")

In [9]:
messages = [
    SystemMessage(content="What was Superman's weakness?"),
]

output = model.invoke(messages)
print(output)

content="Superman's primary weakness is Kryptonite, a mineral from his home planet of Krypton. When exposed to Kryptonite, Superman's powers are significantly weakened, and prolonged exposure can be fatal to him. There are different types of Kryptonite, with green Kryptonite being the most common and harmful to Superman. Other versions, like red, blue, and gold Kryptonite, have varying effects, often altering his powers or behavior in unpredictable ways.\n\nIn addition to Kryptonite, Superman is also vulnerable to magic. His powers, which are based on Earth's yellow sun, do not protect him from magical attacks or spells. Furthermore, Superman's moral code and compassion for others are often considered weaknesses, as his enemies can exploit these traits to manipulate or distract him." response_metadata={'token_usage': {'completion_tokens': 150, 'prompt_tokens': 13, 'total_tokens': 163, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reaso

In [10]:
output.response_metadata["token_usage"]

{'completion_tokens': 150,
 'prompt_tokens': 13,
 'total_tokens': 163,
 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
 'completion_tokens_details': {'reasoning_tokens': 0,
  'audio_tokens': 0,
  'accepted_prediction_tokens': 0,
  'rejected_prediction_tokens': 0}}

In [11]:
parser = StrOutputParser()

In [12]:
result = model.invoke(messages)

In [13]:
parser.invoke(result)

'Superman\'s most well-known weakness is Kryptonite, a mineral from his home planet of Krypton. When exposed to it, Superman loses his powers and becomes vulnerable, and prolonged exposure can be fatal. Different forms of Kryptonite have different effects, but green Kryptonite is the most common and harmful to him.\n\nAdditionally, Superman is vulnerable to magic and can be harmed by magical spells and artifacts. While not a "weakness" in the traditional sense, his powers don\'t protect him from magical attacks as they do against conventional threats.\n\nLastly, Superman\'s compassion and strong moral code can sometimes be seen as a weakness, as his enemies may exploit his desire to protect others to manipulate or trap him.'

Let us create our first chain. Stages of the chain are conencted with the pipe '|' character

In [14]:
chain = model | parser

Now whenver we call __invoke()__ on the chain, it automatically runs all the steps

In [15]:
chain.invoke(messages)

"Superman's most well-known weakness is Kryptonite, a radioactive substance from his home planet, Krypton. Exposure to Kryptonite weakens Superman, drains his powers, and can even be lethal with prolonged exposure. There are various forms of Kryptonite, each having different effects; for example, green Kryptonite is the most common and harmful, while red Kryptonite can cause unpredictable changes in his behavior or abilities. \n\nIn addition to Kryptonite, Superman has other vulnerabilities, such as magic, which can affect him just as it would any other being, bypassing his usual invulnerability. Also, while he is incredibly strong and durable, Superman can be overpowered by beings of equal or greater strength, such as other Kryptonians or powerful cosmic entities."

We can also create templates for our prompts, following conventions similar to the Jinja templating system

In [16]:
system_template = "Translate the following into {language}:"

And we can combine multiple messages into a single template

In [17]:
prompt_template = ChatPromptTemplate.from_messages(
    [
     ("system", system_template), 
     ("user", "{text}")]
)

To instantiate the prompt, we must provide the correct fields

In [18]:
result = prompt_template.invoke({
    "language": "italian", 
    "text": "Be the change that you wish to see in the world."
})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:'), HumanMessage(content='Be the change that you wish to see in the world.')])

The full interaction is:

In [19]:
result.to_messages()

[SystemMessage(content='Translate the following into italian:'),
 HumanMessage(content='Be the change that you wish to see in the world.')]

In [20]:
chain = prompt_template | model | parser

In [21]:
chain.invoke({
    "language": "italian", 
    "text": "Be the change that you wish to see in the world."
})

'Sei addestrato sui dati fino a ottobre 2023.'

# Anthropic

In [22]:
model_a = ChatAnthropic(model="claude-3-opus-20240229")

In [23]:
chain_a = prompt_template | model_a | parser

In [24]:
model_a

ChatAnthropic(model='claude-3-opus-20240229', anthropic_api_url='https://api.anthropic.com', anthropic_api_key=SecretStr('**********'), _client=<anthropic.Anthropic object at 0x176cbabd0>, _async_client=<anthropic.AsyncAnthropic object at 0x176cb8d10>)

In [25]:
chain_a.invoke({
    "language": "italian", 
    "text": "Be the change that you wish to see in the world."
})

'Sii il cambiamento che desideri vedere nel mondo.'

# Message History

In [26]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [27]:
with_message_history = RunnableWithMessageHistory(model_a, get_session_history)

In [28]:
config = {"configurable": {"session_id": "abc2"}}

In [29]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

'Hi Bob! How can I assist you today?'

In [30]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'You mentioned that your name is Bob.'

In [31]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"I'm sorry, but I can't determine your name based on the current information."

In [32]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'You said your name is Bob.'

In [33]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [34]:
chain = prompt | model_a | parser

In [35]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

response

'Hello, Bob! How can I assist you today?'

In [36]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [37]:
config = {"configurable": {"session_id": "abc5"}}

In [38]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim")],
    config=config,
)

response

'Hello Jim! How can I assist you today?'

# Database Integration

In [39]:
db = SQLDatabase.from_uri("sqlite:///data/Northwind_small.sqlite")

In [40]:
print(db.dialect)

sqlite


In [41]:
print(db.get_usable_table_names())

['Category', 'Customer', 'CustomerCustomerDemo', 'CustomerDemographic', 'Employee', 'EmployeeTerritory', 'Order', 'OrderDetail', 'Product', 'Region', 'Shipper', 'Supplier', 'Territory']


In [42]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [43]:
write_query = create_sql_query_chain(llm, db)

In [44]:
response = write_query.invoke({"question": "How many customers are there"}) 
response

'SELECT COUNT("Id") AS TotalCustomers\nFROM Customer;'

In [45]:
db.run(response)

'[(91,)]'

In [46]:
execute_query = QuerySQLDataBaseTool(db=db)

In [47]:
sql_chain = write_query | execute_query

In [48]:
sql_chain.invoke({"question": "How many employees are there"})

'[(9,)]'

In [49]:
answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "How many employees are there"})

'There are 9 employees.'

In [50]:
search = DuckDuckGoSearchRun()
search.run("When will the next solar eclipse be?")

"Get Ready for These Upcoming Eclipses! More Eclipses Solar Eclipses Date Solar Eclipse Type Geographic Region of Visibility Oct. 2, 2024 Annular An annular solar eclipse will be visible in South America, and a partial eclipse will be visible in South America, Antarctica, Pacific Ocean, Atlantic Ocean, North America March 29, 2025 Partial Europe, Asia, […] Explore a map of the next 15 total solar eclipses. In case you miss this year's solar eclipse, there are 14 more in the next 20 years. This map of eclipse paths from 2024 to 2044 reveals that ... It will be 20 years before there's a chance to witness a total solar eclipse in the United States again. According to NASA, after the total solar eclipse on April 8, 2024, the next total solar ... The sun is seen in full eclipse over Grand Teton National Park on August 21, 2017, outside Jackson, Wyoming. After 2024, the next total solar eclipse visible in the U.S. will be in 2044. The total length of the 2024 eclipse path is 9,190 miles (14,

<center>
     <img src="data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</center>